In [1]:
import os
import random
import math
from sklearn.metrics import mean_squared_error
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sn
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pmdarima as pm 
from sklearn.model_selection import train_test_split
from pmdarima.pipeline import Pipeline

In [2]:
os.getcwd()

'C:\\Users\\niles\\project\\project_work\\notebooks'

In [3]:
# Read data:
#os.chdir("..")
df = pd.read_csv("data.csv")

In [4]:
df.head()

,Time,air_pressure[mmHg],air_temperature[degree celcius],relative_humidity[%],wind_speed[M/S],solar_irridiation[W/m²],total_cloud_cover[from ten],electricity_demand_values[kw],heat_demand_values[kw]
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [5]:
len(df)

70080

In [6]:
df = df.dropna()

In [7]:
len(df)

68792

In [8]:
# Rename Columns:
col_mapping = {"Time": "date",
            "air_pressure[mmHg]": "air_pressure",
            "air_temperature[degree celcius]": "air_temperature",
            "relative_humidity[%]": "relative_humidity",
            "wind_speed[M/S]": "wind_speed",
            "solar_irridiation[W/m²]": "solar_irridiation",
            "total_cloud_cover[from ten]": "total_cloud_cover",
            "electricity_demand_values[kw]": "electricity",
            "heat_demand_values[kw]": "heat"}
df = df.rename(columns = col_mapping)

In [9]:
# Change ts to datetime and set as index:
df["date"] = pd.to_datetime(df["date"],format='%Y-%m-%d')
df.head()

,date,air_pressure,air_temperature,relative_humidity,wind_speed,solar_irridiation,total_cloud_cover,electricity,heat
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [10]:
len(df)

68792

In [11]:
df.head()

,date,air_pressure,air_temperature,relative_humidity,wind_speed,solar_irridiation,total_cloud_cover,electricity,heat
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [12]:
df1 = df[["date",'heat']]

In [13]:
df1 = df1.set_index("date")
df1.head()

,heat
date,
2010-12-01 00:00:00,432.940036
2010-12-01 01:00:00,473.935901
2010-12-01 02:00:00,483.278761
2010-12-01 03:00:00,545.921252
2010-12-01 04:00:00,550.526112


In [14]:
x = df1['heat'].values.reshape(-1, 1) #returns a numpy array
min_max_scaler = StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
df1['heat'] = x_scaled

In [15]:
df1.head()

,heat
date,
2010-12-01 00:00:00,0.537153
2010-12-01 01:00:00,0.667290
2010-12-01 02:00:00,0.696948
2010-12-01 03:00:00,0.895801
2010-12-01 04:00:00,0.910419


In [16]:
davg_df = df1.groupby(pd.Grouper(freq='D')).mean()
davg_df.head()

,heat
date,
2010-12-01,0.073040
2010-12-02,-0.212695
2010-12-03,0.291141
2010-12-04,1.031632
2010-12-05,1.426792


In [17]:
len(davg_df)

2920

In [18]:
dftrain, dftest = train_test_split(davg_df, test_size=0.01)

In [19]:
len(dftest)

30

In [20]:
dftrain, dfvali = train_test_split(dftrain, test_size=0.2)

In [21]:
len(dftrain)

2312

In [22]:
len(dfvali)

578

In [23]:
# best
random.seed(0)

In [24]:
# best
#random.seed(0)
pipeline = Pipeline([
    ("model", pm.AutoARIMA(test='adf',
                           start_p=3, start_q=3, max_p=5, max_q=5, start_P=0, start_Q=0, max_P=5, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', 
                           suppress_warnings=True, random_state=20,n_fits=100))
])
'''
pipeline = Pipeline([
    ("model",pm.AutoARIMA(test='adf',
                           max_p=3, max_q=3, m=7,
                           start_P=1,start_Q=1, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=False))
])
'''
pipeline.fit(dftrain)

Performing stepwise search to minimize aic
 ARIMA(3,1,3)(0,1,0)[12]             : AIC=inf, Time=3.70 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=9465.635, Time=0.06 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=8132.582, Time=0.30 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=2.72 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=8760.294, Time=0.10 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=7860.402, Time=0.61 sec
 ARIMA(1,1,0)(3,1,0)[12]             : AIC=7715.206, Time=1.37 sec
 ARIMA(1,1,0)(4,1,0)[12]             : AIC=7643.205, Time=2.34 sec
 ARIMA(1,1,0)(5,1,0)[12]             : AIC=7585.718, Time=5.17 sec
 ARIMA(1,1,0)(5,1,1)[12]             : AIC=inf, Time=27.10 sec
 ARIMA(1,1,0)(4,1,1)[12]             : AIC=inf, Time=17.71 sec
 ARIMA(0,1,0)(5,1,0)[12]             : AIC=8219.786, Time=3.78 sec
 ARIMA(2,1,0)(5,1,0)[12]             : AIC=7341.788, Time=6.26 sec
 ARIMA(2,1,0)(4,1,0)[12]             : AIC=7404.945, Time=2.85 sec
 ARIMA(2,1,0)(5,1,1)[12]             

Pipeline(steps=[('model',
                 AutoARIMA(D=1, d=1, error_action='warn', m=12, max_P=5,
                           max_Q=10, n_fits=100, random_state=20, start_P=0,
                           start_Q=0, start_p=3, start_q=3, test='adf',
                           trace=True))])

In [25]:
##### univariate arima model predict electricity_demand_values  
# best
#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=5, start_P=0, start_Q=0, max_P=5, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, random_state=20, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=0, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, random_state=20, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=0, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='bic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=1, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='bic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, n_fits=100)


In [ ]:
Sample_predictions= pipeline.predict_in_sample(exogenous=None, return_conf_int=False)
Actuals = pd.Series.to_numpy(dftrain['heat'])
Residuals = Actuals-Sample_predictions

In [ ]:
Residuals

In [26]:
forecast, conf = pipeline.predict(n_periods=578, return_conf_int=True)
forecast

array([ 5.08582589e-01,  7.34276003e-01, -6.57015328e-02, -3.82206796e-01,
        2.60285872e-01, -6.31488974e-02,  1.73328444e-01, -3.91353077e-01,
        1.40967253e-01, -7.46301156e-01, -2.79605885e-02, -5.31567497e-02,
        3.75226008e-01,  4.05922467e-01,  4.80942442e-02, -3.05494086e-01,
        1.48313510e-02, -1.67659390e-01,  1.95420178e-01, -3.06020149e-01,
        1.52211299e-01, -7.88611949e-01, -4.69964116e-01,  3.68254706e-02,
        3.99819662e-01,  3.41095351e-01,  8.96520559e-02, -3.23228997e-01,
        1.05705088e-01, -2.12791084e-02,  3.14242972e-01, -3.44796371e-01,
        2.44434847e-01, -7.50954961e-01, -4.74175690e-01,  1.70535164e-01,
        3.19897135e-01,  6.49681485e-01,  3.44165931e-01, -3.45427406e-01,
        3.63759534e-01, -2.94610414e-01,  3.51429317e-01, -4.84093190e-01,
        2.45778266e-01, -8.61097906e-01, -3.49901645e-01,  5.29734790e-04,
        9.57415069e-02,  1.89729819e-01,  9.05200104e-03, -3.94119512e-01,
        1.39402791e-01, -

In [27]:
Actuals = pd.Series.to_numpy(dfvali['heat'])
Actuals

array([ 1.62401518, -0.3399964 ,  0.74293014, -0.59047699, -0.00374984,
        1.50214104,  0.82619097, -0.8371743 , -0.80484678,  0.06553326,
       -0.83669993, -0.10352178,  0.88804356, -0.24020031,  2.96533382,
       -0.8371743 , -0.8371743 , -0.70039001, -0.52657012, -0.35007652,
       -0.82751886, -0.50552461,  2.1196588 , -0.48829302,  0.54918693,
       -0.8371743 , -0.79040357, -0.26037399, -0.8371743 , -0.8371743 ,
       -0.79474864, -0.79474864, -0.82405577,  0.05195392,  0.61984156,
       -0.38802266, -0.69374272,  1.23926114,  0.91836123, -0.36647787,
        2.55134983, -0.43321224, -0.3523514 ,  1.21274646,  2.42881768,
       -0.0206813 , -0.8371743 , -0.27188856,  1.67224696, -0.78221691,
       -0.53709979,  0.50162629, -0.42689045, -0.83663858, -0.8371743 ,
        0.17892618, -0.8371743 , -0.8371743 , -0.82828664, -0.14897713,
        1.85126249, -0.8371743 , -0.55436326,  0.37308754, -0.36302634,
       -0.8371743 , -0.83004773, -0.8371743 , -0.8371743 , -0.16

In [28]:
mseVali = mean_squared_error(Actuals, forecast)
mseVali

2.047911935446724

In [29]:
forecasttrain, _ = pipeline.predict(n_periods=2312, return_conf_int=True)
forecasttrain

array([ 0.50858259,  0.734276  , -0.06570153, ..., -7.28236589,
       -6.60326782, -7.51461242])

In [30]:
Actualstrain = pd.Series.to_numpy(dftrain['heat'])
Actualstrain

array([-0.50509391, -0.47720663, -0.05997899, ..., -0.66210303,
       -0.6442025 , -0.22621699])

In [31]:
msetrain = mean_squared_error(Actualstrain, forecasttrain)
msetrain

17.627104247507216

In [34]:
forecasttest, conf = pipeline.predict(n_periods=30, return_conf_int=True)
forecasttest

array([ 0.50858259,  0.734276  , -0.06570153, -0.3822068 ,  0.26028587,
       -0.0631489 ,  0.17332844, -0.39135308,  0.14096725, -0.74630116,
       -0.02796059, -0.05315675,  0.37522601,  0.40592247,  0.04809424,
       -0.30549409,  0.01483135, -0.16765939,  0.19542018, -0.30602015,
        0.1522113 , -0.78861195, -0.46996412,  0.03682547,  0.39981966,
        0.34109535,  0.08965206, -0.323229  ,  0.10570509, -0.02127911])

In [32]:
Actualstest = pd.Series.to_numpy(dftest['heat'])
Actualstest

array([-0.72381951, -0.71892949, -0.50589547,  0.80078397,  0.53715321,
       -0.81506147, -0.43359816,  2.22110776,  0.15020233, -0.24618141,
        0.66961704,  0.81082455, -0.75549104, -0.8371743 ,  2.10288264,
        2.38166301,  0.5952254 , -0.8371743 , -0.37893297,  0.02716322,
       -0.39414123, -0.60311315, -0.83279186, -0.8371743 , -0.43049864,
        1.7099714 , -0.51617881, -0.8371743 , -0.7580584 , -0.80420663])

In [35]:
msetest = mean_squared_error(Actualstest, forecasttest)
msetest

1.193905848655821

In [36]:
rmse = math.sqrt(msetest)
print('RMSE: %f' % rmse)

RMSE: 1.092660
